In [1]:
# set cuda visible devices
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

import os
if is_notebook():
    os.environ["CUDA_VISIBLE_DEVICES"] = "" #"1"
    # os.environ['CUDA_LAUNCH_BLOCKING']="1"
    # os.environ['TORCH_USE_CUDA_DSA'] = "1"

import matplotlib 
if not is_notebook():
    matplotlib.use('Agg')

# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
# for now incomplete spurious correlation: 
#   waterbirds

# other notebooks:
# multi-class classification: 
#   multi-nli-cc

# known group labels
# waterbirds (normal)
# mulit-nli cc 


In [3]:
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from itertools import product
import json

import numpy as np

from losses.loss_types import LossType
from utils.exp_utils import get_executor, run_experiments

In [4]:
SCRIPT_NAME = "spur_corr_exp.py"
EXP_DIR = "output/real_data_group_labels_aux_weight_sweep"
n_trials = 32

In [10]:
method_configs = {
    "DivDis": {"loss_type": LossType.DIVDIS},
    "TopK 0.1": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.1},
    "TopK 0.5": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.5},
    "DBAT": {"loss_type": LossType.DBAT, "shared_backbone": False, "freeze_heads": True, "binary": True, "batch_size": 16, "target_batch_size": 32},
}

dataset_configs = {
    "waterbirds": {"dataset": "waterbirds", "model": "Resnet50", "epochs": 5, "source_cc": False, "use_group_labels": True},
}

method_ranges = {
    "DivDis": {"aux_weight": [0, 1]},
    "TopK 0.1": {"aux_weight": [0, 1]},
    "TopK 0.5": {"aux_weight": [-1, 1]},
    "DBAT": {"aux_weight": [-1, 1]},
}

configs = {
    (ds_name, method_name): {**ds_config, **method_config} 
    for (ds_name, ds_config), (method_name, method_config) in product(dataset_configs.items(), method_configs.items())
}


sampled_configs = []
for (ds_name, method_name), conf in configs.items():
    for i in range(n_trials):
        sample_range = method_ranges[method_name]["aux_weight"]
        aux_weight = 10**(np.random.uniform(sample_range[0], sample_range[1]))
        seed = np.random.randint(0, 1000000)
        conf["aux_weight"] = aux_weight
        conf["seed"] = seed
        sampled_configs.append(conf)

def get_conf_exp_dir(ds_name, method_name, aux_weight):
    return Path(EXP_DIR, f"{ds_name}_{method_name}/{aux_weight}")

for (ds_name, method_name), conf in configs.items():
    exp_dir = get_conf_exp_dir(ds_name, method_name, conf["aux_weight"])
    conf["exp_dir"] = exp_dir
    
for conf in configs.values():
    conf["plot_activations"] = False

In [ ]:
executor = get_executor(exp_dir, mem_gb=32)
jobs = run_experiments(sampled_configs, executor, SCRIPT_NAME)